In [ ]:
import pandas as pd
import random
from fuzzywuzzy import process
from collections import Counter

# Veri setlerinin dosya yolları
movies_file_path = '/content/movie.csv'  # movies.csv dosyasının yolu
ratings_file_path = '/content/rating.csv'  # ratings.csv dosyasının yolu

# MovieLens veri setini yükleyelim
movies_df = pd.read_csv(movies_file_path)
ratings_df = pd.read_csv(ratings_file_path)

# Film türlerine göre rastgele film önerisi yapan fonksiyon
def recommend_movies_by_genre():
    genre = input("Lütfen bir film türü girin (örneğin: Drama, Comedy, Action): ").lower()
    genre_movies = movies_df[movies_df['genres'].str.contains(genre, case=False, na=False)]

    if genre_movies.empty:
        return f"{genre.capitalize()} türünde film bulunamadı."

    random_movie = genre_movies.sample(1)
    return f"Rastgele {genre} filmi önerisi: {random_movie.iloc[0]['title']}"

# Belirli bir puan aralığındaki filmleri öneren fonksiyon
def recommend_movies_by_rating():
    min_rating = float(input("Minimum puanı girin (1.0 ile 5.0 arasında): "))
    max_rating = float(input("Maksimum puanı girin (1.0 ile 5.0 arasında): "))

    # MovieLens veri setinde her film için puan ortalamalarını hesaplıyoruz
    avg_ratings = ratings_df.groupby('movieId')['rating'].mean()

    # Puanları filtreliyoruz
    filtered_movies = movies_df[movies_df['movieId'].isin(avg_ratings[(avg_ratings >= min_rating) & (avg_ratings <= max_rating)].index)]

    if filtered_movies.empty:
        return f"{min_rating} ile {max_rating} arasında puanlanan film bulunamadı."

    random_movie = filtered_movies.sample(1)
    return f"Rastgele {min_rating}-{max_rating} arası puanlanan film önerisi: {random_movie.iloc[0]['title']}"

# Kullanıcının istediği filmin temel bilgilerini veren fonksiyon
def get_movie_details():
    movie_title = input("Lütfen film adını girin: ").strip()  # Kullanıcı girdisini temizle

    # Filmi içeren tüm satırları filtrele
    movie = movies_df[movies_df['title'].str.contains(movie_title, case=False, na=False, regex=False)]

    if movie.empty:
        return "Film bulunamadı. Lütfen ismi doğru girdiğinizden emin olun."

    # İlk eşleşen filmi al
    movie_details = movie.iloc[0]

    return (f"\n🎬 **Film Adı ve Yılı:** {movie_details['title']}\n"
            f"📌 **Tür:** {movie_details['genres']}\n"
            f"🗓️ **Yıl:** {movie_details['year']}")  # Yıl bilgisini de ekledim

# Kullanıcının belirlediği yılda çıkan filmleri listeleyen fonksiyon (EN FAZLA 20 FİLM)
def recommend_movies_by_year():
    year = input("Lütfen bir yıl girin: ")  # Kullanıcıdan yıl bilgisi al
    year_movies = movies_df[movies_df['title'].str.contains(str(year), case=False, na=False)]  # Yılı içeren filmleri filtrele

    if year_movies.empty:
        return f"{year} yılında çıkan film bulunamadı."

    # En fazla 20 film göster
    limited_movies = year_movies['title'].head(20).tolist()

    return f"{year} yılında çıkan filmler (max 20):\n" + "\n".join(limited_movies)

# Kullanıcının sevdiği 3 filmi girip öneri alması fonksiyonu
def recommend_by_favorite_movies():
    print("Lütfen tam olarak 3 sevdiğiniz filmi girin.")

    # Kullanıcıdan 3 film alın
    fav_movies = []
    for i in range(3):
        fav_movie = input(f"{i+1}. Film: ").strip()  # Kullanıcıdan film adı al
        fav_movies.append(fav_movie)

    # Film başlıkları arasından en iyi eşleşmeleri bulmak için fuzzywuzzy kullanıyoruz
    matched_movies = []
    for movie in fav_movies:
        # En yakın 3 eşleşmeyi al
        matches = process.extract(movie, movies_df['title'], limit=3)
        matched_movies.extend([match[0] for match in matches if match[1] > 70])  # 70'ten büyük eşleşmeleri al

    matched_movies = list(set(matched_movies))  # Yinelenenleri kaldır

    if not matched_movies:
        return "Seçtiğiniz filmler bulunamadı. Lütfen doğru yazdığınızdan emin olun."

    # Seçilen filmlerin türlerini topla
    selected_movies = movies_df[movies_df['title'].isin(matched_movies)]

    # Seçilen filmlerin türlerini topla
    genres_list = selected_movies['genres'].str.split('|').explode().tolist()

    # En sık geçen türü belirle
    most_common_genre = Counter(genres_list).most_common(1)[0][0]  # En çok geçen tür

    # Bu türde başka filmler öner
    recommended_movies = movies_df[movies_df['genres'].str.contains(most_common_genre, case=False, na=False)]

    if recommended_movies.empty:
        return f"Üzgünüm, {most_common_genre} türünde başka film bulunamadı."

    # İlk 3 önerilen filmi al
    recommended_movies_list = recommended_movies['title'].head(3).tolist()

    # Her öneriyi 1. Film, 2. Film, 3. Film olarak listele
    recommendations = "\n".join([f"{i+1}. Film: {recommended_movies_list[i]}" for i in range(len(recommended_movies_list))])

    return (f"🎥 Sevdiğiniz filmlere göre öneriler:\n" + recommendations)


# Ana menü fonksiyonu
def main():
    while True:
        print("\nFilm Rehberine Hoş Geldiniz! Lütfen bir seçenek seçin:")
        print("1 - Film türüne göre öneri al")
        print("2 - Puan aralığına göre rastgele film önerisi yap")
        print("3 - Filmin temel bilgilerini al")
        print("4 - Belirtilen yılda çıkan filmleri göster (max 20)")
        print("5 - Sevdiğiniz filmlerle öneri al")
        print("6 - Çıkış")

        choice = input("Seçiminizi girin (1/2/3/4/5/6): ")

        if choice == "1":
            print(recommend_movies_by_genre())
        elif choice == "2":
            print(recommend_movies_by_rating())
        elif choice == "3":
            print(get_movie_details())
        elif choice == "4":
            print(recommend_movies_by_year())
        elif choice == "5":
            print(recommend_by_favorite_movies())
        elif choice == "6":
            print("Çıkış yapılıyor...")
            break
        else:
            print("Geçersiz seçim, lütfen tekrar deneyin.")

if __name__ == "__main__":
    main()
